In [1]:
import os
import faiss
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.core import Settings

ModuleNotFoundError: No module named 'llama_index'

In [2]:
docs = SimpleDirectoryReader(input_dir="./data").load_data()

In [3]:
docs[0].__dict__

{'id_': 'fbd122e4-d08e-419e-beda-10d7be0fadef',
 'embedding': None,
 'metadata': {'file_path': '/mnt/d/DEV/MOC-Training/data/sample_doc_thai_1.txt',
  'file_name': 'sample_doc_thai_1.txt',
  'file_type': 'text/plain',
  'file_size': 955,
  'creation_date': '2025-07-15',
  'last_modified_date': '2025-07-15'},
 'excluded_embed_metadata_keys': ['file_name',
  'file_type',
  'file_size',
  'creation_date',
  'last_modified_date',
  'last_accessed_date'],
 'excluded_llm_metadata_keys': ['file_name',
  'file_type',
  'file_size',
  'creation_date',
  'last_modified_date',
  'last_accessed_date'],
 'relationships': {},
 'metadata_template': '{key}: {value}',
 'metadata_separator': '\n',
 'text_resource': MediaResource(embeddings=None, data=None, text='ปัญญาประดิษฐ์ (AI) ในประเทศไทยกำลังพัฒนาอย่างรวดเร็ว หน่วยงานทั้งภาครัฐและเอกชนให้ความสำคัญกับการนำ AI ไปประยุกต์ใช้ในหลายด้าน เช่น การแพทย์ การศึกษา การเกษตร และอุตสาหกรรม\n\nหนึ่งในหัวข้อที่ได้รับความสนใจมากคือ การนำโมเดลภาษา (LLM) มาช่วยในการ

In [5]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline

# ตั้งค่าให้แยกเอกสารเป็น chunk ย่อย ๆ
text_splitter = SentenceSplitter(
    separator=" ",       # ใช้เว้นวรรคเป็นตัวแยก
    chunk_size=128,     # ขนาดของแต่ละ chunk (จำนวนตัวอักษร)
    chunk_overlap=12    # ความซ้อนของข้อความระหว่างแต่ละ chunk
)

# สร้าง ingestion pipeline ที่ประกอบด้วย text_splitter
pipeline = IngestionPipeline(
    transformations=[
        text_splitter,
    ]
)

# รัน pipeline กับเอกสารที่โหลดไว้ เพื่อแยกเป็น chunks
nodes = pipeline.run(
    documents=docs,
    in_place=True,       # แก้ไขเอกสารเดิมแทนที่จะสร้างใหม่
    show_progress=True,  # แสดง progress bar ขณะรัน
)

Parsing nodes:   0%|          | 0/3 [00:00<?, ?it/s]

In [6]:


# dimensions of BAAI/bge-m3
d = 1024
faiss_index = faiss.IndexFlatIP(d)

vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [7]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embedding_model_name = 'BAAI/bge-m3'
embed_model = HuggingFaceEmbedding(model_name=embedding_model_name,max_length=1024, device='cuda:0')

In [9]:
from llama_index.core import Settings

Settings.llm = None
Settings.embed_model = embed_model

LLM is explicitly disabled. Using MockLLM.


In [10]:
index = VectorStoreIndex(
    nodes=nodes, storage_context=storage_context, show_progress=True,  # แสดง progress bar ขณะรัน
)

Generating embeddings:   0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
index = VectorStoreIndex.from_documents(docs, embed_model=embed_model)


In [12]:
retriever = index.as_retriever(similarity_top_k=5)
response = retriever.retrieve('LlamaIndex ')
for i in range(5):
    print('Response:', response[i].text,'\n')
    print('Score:', response[i].score)
    print('-'*50,'\n\n')

KeyError: '7'

In [ ]:
from llama_index.core import Settings

Settings.llm = None
Settings.embed_model = embed_model

In [ ]:
from llama_index.core import (

    load_index_from_storage,
    StorageContext,
)

from IPython.display import Markdown, display
from llama_index.vector_stores.faiss import FaissVectorStore

In [ ]:
persist_dir="./storage/"
vector_store = FaissVectorStore.from_persist_dir(persist_dir)
storage_context = StorageContext.from_defaults(vector_store=vector_store, persist_dir=persist_dir)
index = load_index_from_storage(storage_context=storage_context)

In [ ]:
retriever = index.as_retriever(similarity_top_k=5)
response = retriever.retrieve('AI สำคัญกับประเทศไทยอย่างไร')
for i in range(5):
    print('Response:', response[i].text,'\n')
    print('Score:', response[i].score)
    print('-'*50,'\n\n')